# Plotting CapCruncher output

## Introduction

This is a quick overview of the plotting functionality of CapCruncher. Please ensure that you have installed the optional dependencies for CapCruncher by running:

``` bash
pip install capcruncher[plotting]
```

<!-- For more detailed information on the plotting functionality, see the [plotting guide](plotting.md). -->

## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import coolbox.api as cb
from capcruncher.api.plotting import CCTrack, CCFigure
import pyranges as pr

## Read plotting coordinates

In [ ]:
viewpoints = pr.read_bed("oligoCoordinates.bed")
viewpoints

## Plot using the CapCruncher API

First create a number of `CCTrack` instances supported track types:

- heatmap - a contact matrix heatmap in cool format 
- bigwig  - a bigwig file containing the number of reads per bin
- bigwig_summary - a collection of bigwig files containing the number of reads per bin
- scale - a scale bar. Does not require a file to be specified
- bed  - a bed file
- xaxis  - an x-axis of genomic coordinates. Does not require a file to be specified
- genes - a gene track in bed12 format 
- spacer - a spacer track. Does not require a file to be specified


## Create new `CCTrack` objects

### Option 1: Create a list of `CCTrack` objects and pass them to CCFigure

In [ ]:
tracks = [
    CCTrack(None, type="scale"),
    CCTrack(
        "capcruncher_output/GM12878_rep1/GM12878.hdf5",
        type="heatmap",
        binsize=5000,
        title="GM12878",
        viewpoint="BLC2",
        normalization="ice",
        transform="yes",
        style="triangular",
    ),
    CCTrack(None, type="spacer"),
    CCTrack("hg38_genes.bed", type="genes", color="black"),
    CCTrack(None, type="spacer"),
    CCTrack(None, type="xaxis"),
]

fig = CCFigure(tracks, auto_spacing=False)

### Option 2: Create a `CCFigure` object and add tracks to it

In [ ]:
fig = CCFigure(tracks, auto_spacing=False)
fig.add_track(CCTrack("hg38_genes.bed", type="genes", color="black"))

## Plot the figure (Optional)

In [ ]:

# Plot a specific region if desired
fig.plot("chr18:61460000-64500000")

## Save the figure

Two options: 

1. Save the figure as a static image using the save method of the `CCFigure`.
2. Save the `CCFigure` as a TOML file which can be edited and either reloaded into a `CCFigure` or used by the command line interface to generate a new figure using `capcruncher plot make-plot`.


### Option 1: Save the figure to a file 

In [ ]:
fig.save("chr18:61460000-64500000", output="test.png")

### Option 2: Save the figure as a template

In [ ]:
fig.to_toml("template.toml")

This will look something like this:

``` TOML

```

This template can either be re-loaded e.g.

In [ ]:
fig = CCFigure.from_toml("template.toml")

Alternatively the template can be edited and used with `capcruncher plot make-plot` e.g.

In [ ]:
%%bash

capcruncher \
plot \
make-plot \
--template template.toml \
--region chr18:61460000-64500000 \
-o test.pdf

